In [1]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input, Concatenate
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils, plot_model
from keras.layers.merge import concatenate

Using TensorFlow backend.


In [2]:
np.random.seed(2)

(X_train, y_train),(X_test, y_test) = mnist.load_data()

#tensorflow (no_of_samples, height, width, channels)
#theano (no_of_samples, channels, height, width)

X_train = X_train.reshape(X_train.shape[0],28,28,1).astype('float32')
X_test = X_test.reshape(X_test.shape[0],28,28,1).astype('float32')

X_train = X_train/255
X_test = X_test/255

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [11]:
'''
print(y_train.shape) #(60000,10)
print(y_train[0]) #[0,0,0,0,1,0,0,0,0,0]
'''

def base_model():
    model = Sequential()
    model.add(Conv2D(4,(3,3), input_shape=(28,28,1),activation='relu' ))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
    return model

In [12]:
model = base_model()
plot_model(model, to_file='default_CNN.png')
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 26, 26, 4)         40        
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 4)         0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 13, 13, 4)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 676)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                6770      
_________________________________________________________________
dense_6 (Dense)              (None, 10)                110       
Total params: 6,920
Trainable params: 6,920
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=10, batch_size=128, verbose=1)
print(model.evaluate(X_test,y_test,verbose=2)[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 11s 176us/step - loss: 0.7506 - acc: 0.7648 - val_loss: 0.3189 - val_acc: 0.9087
Epoch 2/10
60000/60000 [==============================] - 10s 171us/step - loss: 0.3247 - acc: 0.9037 - val_loss: 0.2177 - val_acc: 0.9356
Epoch 3/10
60000/60000 [==============================] - 10s 171us/step - loss: 0.2443 - acc: 0.9268 - val_loss: 0.1656 - val_acc: 0.9511
Epoch 4/10
60000/60000 [==============================] - 10s 171us/step - loss: 0.2059 - acc: 0.9378 - val_loss: 0.1423 - val_acc: 0.9576
Epoch 5/10
60000/60000 [==============================] - 10s 172us/step - loss: 0.1875 - acc: 0.9432 - val_loss: 0.1265 - val_acc: 0.9628
Epoch 6/10
60000/60000 [==============================] - 10s 171us/step - loss: 0.1714 - acc: 0.9488 - val_loss: 0.1178 - val_acc: 0.9651
Epoch 7/10
60000/60000 [==============================] - 10s 171us/step - loss: 0.1638 - acc: 0.9504 - val_loss: 0.

In [14]:
def functional_model():
    # input layer
    visible = Input(shape=(28,28,1))
    
    conv1 = Conv2D(2, kernel_size=(3,3), activation='relu')(visible)
    #pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    #flat1 = Flatten()(pool1)
    
    conv2 = Conv2D(1, kernel_size=(3,3), activation='relu',trainable=False)(visible)
    #pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    #flat2 = Flatten()(pool2)
    
    conv3 = Conv2D(1,kernel_size=(3,3), activation='relu',trainable=False)(visible)
    
    merge = concatenate([conv1, conv2,conv3])
    
    pool1 = MaxPooling2D(pool_size=(2, 2))(merge)
    dropout1 = Dropout(0.2)(pool1)
    flatten1 = Flatten()(dropout1)
    # interpretation layer
    hidden1 = Dense(10, activation='relu')(flatten1)
    # prediction output
    output = Dense(10, activation='sigmoid')(hidden1)
    model = Model(inputs=visible, outputs=output)
    return model

In [20]:
custom_model = functional_model()
print(custom_model.summary())
plot_model(custom_model, to_file='custom_CNN.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 26, 26, 2)    20          input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 26, 26, 1)    10          input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 26, 26, 1)    10          input_3[0][0]                    
__________________________________________________________________________________________________
concatenat

In [16]:
print("Weights before change:")
print (custom_model.layers[2].get_weights())
print (custom_model.layers[3].get_weights())

Weights before change:
[array([[[[ -3.29269111e-01]],

        [[  3.23578060e-01]],

        [[ -2.77940005e-01]]],


       [[[ -4.28223252e-01]],

        [[ -4.92406487e-01]],

        [[  2.32425153e-01]]],


       [[[  1.44603252e-01]],

        [[  5.00469327e-01]],

        [[  1.68085098e-04]]]], dtype=float32), array([ 0.], dtype=float32)]
[array([[[[ 0.55533087]],

        [[ 0.03697348]],

        [[-0.40264907]]],


       [[[ 0.46966326]],

        [[-0.2204718 ]],

        [[ 0.21844119]]],


       [[[-0.57704937]],

        [[-0.05162603]],

        [[ 0.49214005]]]], dtype=float32), array([ 0.], dtype=float32)]


In [17]:
#setting custom weights for layers
laplacian = [np.asarray([[[[1]],

        [[1 ]],

        [[1]]],


       [[[1]],

        [[-8]],

        [[1]]],


       [[[1]],

        [[1]],

        [[1]]]], dtype='float32'), np.asarray([ 0.], dtype='float32')]

mean_filter = [np.asarray([[[[.111]],

        [[.111 ]],

        [[.111]]],


       [[[.111]],

        [[.111]],

        [[.111]]],


       [[[.111]],

        [[.111]],

        [[.111]]]], dtype='float32'), np.asarray([ 0.], dtype='float32')]


custom_model.layers[2].set_weights(laplacian)
custom_model.layers[3].set_weights(mean_filter)

#print (custom_model.layers[2].get_weights())
#print (custom_model.layers[3].get_weights())

In [19]:
custom_model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
custom_model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=10, batch_size=128, verbose=1)
print(custom_model.evaluate(X_test,y_test,verbose=2)[1]*100)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 18s 302us/step - loss: 0.2731 - acc: 0.9170 - val_loss: 0.2124 - val_acc: 0.9359
Epoch 2/10
60000/60000 [==============================] - 20s 330us/step - loss: 0.2585 - acc: 0.9214 - val_loss: 0.1983 - val_acc: 0.9439
Epoch 3/10
60000/60000 [==============================] - 20s 334us/step - loss: 0.2513 - acc: 0.9229 - val_loss: 0.1906 - val_acc: 0.9452
Epoch 4/10
60000/60000 [==============================] - 20s 334us/step - loss: 0.2364 - acc: 0.9281 - val_loss: 0.1794 - val_acc: 0.9504
Epoch 5/10
60000/60000 [==============================] - 22s 366us/step - loss: 0.2239 - acc: 0.9312 - val_loss: 0.1675 - val_acc: 0.9507
Epoch 6/10
60000/60000 [==============================] - 23s 379us/step - loss: 0.2194 - acc: 0.9324 - val_loss: 0.1615 - val_acc: 0.9523
Epoch 7/10
60000/60000 [==============================] - 23s 389us/step - loss: 0.2089 - acc: 0.9357 - val_loss: 0.